In [7]:
# third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from preprocessing_functions import *

In [8]:
dataSource = 'immonet_data/raw_data.csv'
outlierList = ["livingSpace", "roomCount", "floor", "rentPerParking", "deposit", "base_rent", "parkingCount"]
categoryList = ["condition", "apartmentType", "energyEfficiencyClass", "energyType", "heatingType", "floorType"]
destinationPath = #'immonet_data/raw_data_without_drop.csv'

In [9]:

properties = pd.read_csv(
    dataSource
)

In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [11]:
properties.head()

,_id,observationDate,state,city,AP_community,livingSpace,roomCount,floor,constructionYear,apartmentType,condition,rentPerParking,deposit,base_rent,propertyTransferTax,parkingCount,energyEfficiencyClass,energyConsumption,energyType,heatingType,balcony,terrace,garden,furnished,builtInKitchen,floorHeating,floorType,cableConnection,undergroundParking,outsideParking,cellar,lift,barrierfree,wheelchair,fireplace,longitude,latitude,coord_confidence,distance_kindergarden,distance_school,distance_university,distance_ATM,distance_bakery,distance_DIY_store,distance_hairdresser,distance_supermarket,distance_clinic,distance_doctor,distance_hospital,distance_pharmacy,distance_airport,distance_bus_stop,distance_charging_station,distance_fuel,distance_harbour,distance_recycling_center,distance_train_station,distance_tram_station,distance_bar,distance_beergarden,distance_nightclub,distance_restaurant,distance_children,distance_seniors,distance_shelter,distance_hotel,distance_museum
0,6314c2df2ddc363b16f197b8,2022-09-13T01:31:33.368285,Schleswig-Holstein,Grömitz,Kreis Ostholstein,54.00,2.0,1.0,1975.0,Apartment,TEIL_VOLLRENOVIERT,50.0,NaN,1000.00,6.5,1.0,NaN,174 kWh/(m²*a),Gas,Zentralheizung,True,False,False,False,True,False,NaN,False,False,False,True,True,True,True,False,10.951870,54.140680,9.0,636.0,521.0,36938.0,643.0,912.0,1134.0,1103.0,1293.0,9288.0,1044.0,9779.0,989.0,40779.0,457.0,419.0,550.0,33713.0,6245.0,9627.0,61018.0,859.0,2972.0,17049.0,17.0,29728.0,1453.0,62983.0,89.0,5664.0
1,6314c2df2ddc363b16f197b9,2022-09-27T00:42:52.757741,Schleswig-Holstein,Barsbüttel,Kreis Stormarn,82.00,3.0,NaN,1951.0,Wohnung,TEIL_RENOVIERUNGSBED,NaN,2385.00,795.00,6.5,NaN,NaN,NaN,Gas,NaN,False,True,False,False,True,False,NaN,False,False,False,True,False,False,False,False,10.169839,53.568982,9.0,196.0,486.0,3909.0,48.0,340.0,2524.0,44.0,384.0,2103.0,69.0,2049.0,350.0,13144.0,48.0,153.0,1123.0,15763.0,1322.0,4121.0,127734.0,3616.0,7862.0,1760.0,315.0,2061.0,1581.0,1990.0,3277.0,3565.0
2,6314c2df2ddc363b16f197ba,2022-09-24T00:34:43.274553,Schleswig-Holstein,Ellerbek,Kreis Pinneberg,90.00,2.5,1.0,2020.0,Maisonette,TEIL_VOLLRENOVIERT,NaN,2940.00,980.00,6.5,1.0,A+,15 kWh/(m²*a),Erdwärme,Zentralheizung,True,False,False,False,True,True,NaN,False,False,False,True,False,False,False,False,9.900665,53.674481,0.0,755.0,799.0,8625.0,1162.0,1092.0,4414.0,1209.0,1104.0,6230.0,1175.0,4868.0,1149.0,8241.0,1250.0,3498.0,1465.0,15583.0,5279.0,1386.0,85655.0,3083.0,4383.0,8837.0,944.0,12056.0,775.0,3205.0,2689.0,824.0
3,6314c2df2ddc363b16f197bb,2022-09-25T02:02:09.124346,Schleswig-Holstein,Elmenhorst,Kreis Stormarn,120.00,4.0,1.0,2022.0,Wohnung,Neubau,30.0,4500.00,1500.00,6.5,2.0,A+,17 kWh/(m²*a),Luft-/Wasserwärme,NaN,True,False,False,False,True,False,NaN,False,False,False,False,False,False,False,False,10.274517,53.764947,0.0,2921.0,3310.0,16113.0,617.0,609.0,9330.0,884.0,615.0,10492.0,3869.0,7757.0,3894.0,22340.0,358.0,3734.0,759.0,31537.0,5171.0,4105.0,72992.0,10410.0,3915.0,4678.0,3851.0,10585.0,3694.0,14472.0,2190.0,4254.0
4,6314c2df2ddc363b16f197bc,2022-09-10T00:00:12.671605,Schleswig-Holstein,Bad Oldesloe,Kreis Stormarn,63.98,2.5,NaN,1998.0,Dachgeschosswohnung,NaN,NaN,1335.12,445.04,6.5,NaN,B,90 kWh/(m²*a),Fernwärme,Zentralheizung,True,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False,10.386690,53.806950,9.0,1037.0,416.0,7391.0,354.0,364.0,1115.0,927.0,184.0,17730.0,737.0,1661.0,397.0,20740.0,177.0,270.0,521.0,39899.0,1817.0,323.0,66762.0,98.0,8499.0,631.0,171.0,14841.0,1260.0,16370.0,375.0,894.0


In [12]:
cleaned_data = properties
for i in range(1):
    outlier_index_list = []    
    for attribute in outlierList:
        outlier_index_list.extend(outliers(cleaned_data, attribute))
    cleaned_data = remove_outliers(cleaned_data, outlier_index_list)

'cleaned_data = properties\nfor i in range(1):\n    outlier_index_list = []    \n    for attribute in outlierList:\n        outlier_index_list.extend(outliers(cleaned_data, attribute))\n    cleaned_data = remove_outliers(cleaned_data, outlier_index_list)'

In [13]:
cleaned_data.drop(cleaned_data[cleaned_data["constructionYear"] > 2022].index, inplace = True)

In [14]:
for category in categoryList:
    cleaned_data[category] = cleaned_data[category].astype("category")
    cleaned_data[category] = cleaned_data[category].cat.codes

In [15]:
cleaned_data["energyConsumption"] = cleaned_data["energyConsumption"].str[:-11]

In [16]:
cleaned_data["energyConsumption"] = cleaned_data["energyConsumption"].astype(float)

In [17]:
cleaned_data.replace({False: 0, True: 1}, inplace = True)

In [18]:
cleaned_data.isna().sum()

_id                              0
observationDate                  0
state                            0
city                             0
AP_community                     8
livingSpace                      0
roomCount                        0
floor                        32911
constructionYear             18809
apartmentType                    0
condition                        0
rentPerParking               74381
deposit                      29819
base_rent                        0
propertyTransferTax           2578
parkingCount                 63774
energyEfficiencyClass            0
energyConsumption            35180
energyType                       0
heatingType                      0
balcony                          0
terrace                          0
garden                           0
furnished                        0
builtInKitchen                   0
floorHeating                     0
floorType                        0
cableConnection                  0
undergroundParking  

In [19]:
cleaned_data["floor"] = cleaned_data["floor"].replace(np.nan, cleaned_data["floor"].median())
cleaned_data["constructionYear"] = cleaned_data["constructionYear"].replace(np.nan, cleaned_data["constructionYear"].median())
cleaned_data["rentPerParking"] = cleaned_data["rentPerParking"].replace(np.nan, cleaned_data["rentPerParking"].median())
cleaned_data["deposit"] = cleaned_data["deposit"].replace(np.nan, cleaned_data["deposit"].median())
cleaned_data["propertyTransferTax"] = cleaned_data["propertyTransferTax"].replace(np.nan, cleaned_data["propertyTransferTax"].median())
cleaned_data["parkingCount"] = cleaned_data["parkingCount"].replace(np.nan, cleaned_data["parkingCount"].median())
cleaned_data["energyConsumption"] = cleaned_data["energyConsumption"].replace(np.nan, cleaned_data["energyConsumption"].median())
cleaned_data["distance_restaurant"] = cleaned_data["distance_restaurant"].replace(np.nan, cleaned_data["distance_restaurant"].median())

In [20]:
cleaned_data = cleaned_data[cleaned_data["AP_community"].notna()]

In [21]:
#cleaned_data = cleaned_data.drop(cleaned_data.index[69016:])

In [22]:
distances = cleaned_data.columns[38:67].values.tolist()
distances.remove("distance_restaurant")

for distance in distances:
    cleaned_data[distance] = cleaned_data[distance].replace(np.nan, cleaned_data[distance].median())


C:\Users\soube\AppData\Local\Temp\ipykernel_4168\1823209243.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[distance] = cleaned_data[distance].replace(np.nan, cleaned_data[distance].median())
C:\Users\soube\AppData\Local\Temp\ipykernel_4168\1823209243.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[distance] = cleaned_data[distance].replace(np.nan, cleaned_data[distance].median())
C:\Users\soube\AppData\Local\Temp\ipykernel_4168\1823209243.py:5: SettingWithCopyWarning: 
A v

In [23]:
cleaned_data.isna().sum()

_id                          0
observationDate              0
state                        0
city                         0
AP_community                 0
livingSpace                  0
roomCount                    0
floor                        0
constructionYear             0
apartmentType                0
condition                    0
rentPerParking               0
deposit                      0
base_rent                    0
propertyTransferTax          0
parkingCount                 0
energyEfficiencyClass        0
energyConsumption            0
energyType                   0
heatingType                  0
balcony                      0
terrace                      0
garden                       0
furnished                    0
builtInKitchen               0
floorHeating                 0
floorType                    0
cableConnection              0
undergroundParking           0
outsideParking               0
cellar                       0
lift                         0
barrierf

In [24]:
for col in cleaned_data.columns:
    if cleaned_data[col].dtype == np.int8:
        cleaned_data[col] = cleaned_data[col].astype(float)
    if cleaned_data[col].dtype == np.int64:
        cleaned_data[col] = cleaned_data[col].astype(float)

C:\Users\soube\AppData\Local\Temp\ipykernel_4168\2118145477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[col] = cleaned_data[col].astype(float)
C:\Users\soube\AppData\Local\Temp\ipykernel_4168\2118145477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[col] = cleaned_data[col].astype(float)


In [25]:
datasets = [cleaned_data]

In [26]:
for dataset in datasets:
    dataset["averageRent_APC"] = dataset["base_rent"].groupby(dataset["AP_community"]).transform("mean")

C:\Users\soube\AppData\Local\Temp\ipykernel_4168\2334878975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["averageRent_APC"] = dataset["base_rent"].groupby(dataset["AP_community"]).transform("mean")


In [27]:
# Normalize the data
"""for dataset in datasets:
    numerical = dataset.select_dtypes(include='float64').columns
    #Normalize the data by MinMaxScaling
    dataset.loc[:,numerical] = preprocessing.MinMaxScaler().fit_transform(dataset.loc[:,numerical])"""

"for dataset in datasets:\n    numerical = dataset.select_dtypes(include='float64').columns\n    #Normalize the data by MinMaxScaling\n    dataset.loc[:,numerical] = preprocessing.MinMaxScaler().fit_transform(dataset.loc[:,numerical])"

In [28]:
for dataset in datasets:
    # Select columns with 'float64' dtype  
    float64_cols = list(dataset.select_dtypes(include='float64'))

    # The same code again calling the columns
    dataset[float64_cols] = dataset[float64_cols].astype('float32')

C:\Users\soube\AppData\Local\Temp\ipykernel_4168\2471394642.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[float64_cols] = dataset[float64_cols].astype('float32')


In [29]:
# Remove all columns without a confidence of 9
cleaned_data_conf = cleaned_data.loc[cleaned_data["coord_confidence"] == 9]

In [30]:
#Export the normalized cleaned Dataframe to csv file
cleaned_data.to_csv(destinationPath, index = False)